In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [22]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Udemy/ML com Python/Datasets/mushroom_dataset.csv')

In [3]:
df.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,mushroom
0,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE
1,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,WHITE,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS,EDIBLE
2,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE
3,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,PINK,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,BROWN,SEVERAL,WOODS,EDIBLE
4,CONVEX,SMOOTH,WHITE,BRUISES,ALMOND,FREE,CROWDED,NARROW,BROWN,TAPERING,BULBOUS,SMOOTH,SMOOTH,WHITE,WHITE,PARTIAL,WHITE,ONE,PENDANT,PURPLE,SEVERAL,WOODS,EDIBLE


In [5]:
df.shape

(8416, 23)

In [6]:
# Verificar a quantidade de valores únicos de cada coluna
df.nunique()

,0
cap-shape,6
cap-surface,4
cap-color,10
bruises,2
odor,9
gill-attachment,2
gill-spacing,2
gill-size,2
gill-color,12
stalk-shape,2


In [8]:
# Forma alternativa de verificar os valores únicos de cada coluna.
[df[c].value_counts() for c in list(df.columns)]

[cap-shape
 CONVEX     3796
 FLAT       3292
 KNOBBED     840
 BELL        452
 SUNKEN       32
 CONICAL       4
 Name: count, dtype: int64,
 cap-surface
 SCALY      3268
 SMOOTH     2684
 FIBROUS    2460
 GROOVES       4
 Name: count, dtype: int64,
 cap-color
 BROWN       2320
 GRAY        2096
 RED         1500
 YELLOW      1072
 WHITE       1040
 BUFF         168
 PINK         144
 CINNAMON      44
 PURPLE        16
 GREEN         16
 Name: count, dtype: int64,
 bruises
 NO         5040
 BRUISES    3376
 Name: count, dtype: int64,
 odor
 NONE        3808
 FOUL        2160
 FISHY        576
 SPICY        576
 ALMOND       400
 ANISE        400
 PUNGENT      256
 CREOSOTE     192
 MUSTY         48
 Name: count, dtype: int64,
 gill-attachment
 FREE        8200
 ATTACHED     216
 Name: count, dtype: int64,
 gill-spacing
 CLOSE      6824
 CROWDED    1592
 Name: count, dtype: int64,
 gill-size
 BROAD     5880
 NARROW    2536
 Name: count, dtype: int64,
 gill-color
 BUFF         1728
 PINK

## Separação das variáveis

In [9]:
# Definindo variáveis preditoras e target
y = df['mushroom']
X = df.drop('mushroom', axis=1)

## Aplicação de OHE

In [12]:
X_encoding = pd.get_dummies(X).astype(int)

In [13]:
X_encoding.head()

,cap-shape_BELL,cap-shape_CONICAL,cap-shape_CONVEX,cap-shape_FLAT,cap-shape_KNOBBED,cap-shape_SUNKEN,cap-surface_FIBROUS,cap-surface_GROOVES,cap-surface_SCALY,cap-surface_SMOOTH,cap-color_BROWN,cap-color_BUFF,cap-color_CINNAMON,cap-color_GRAY,cap-color_GREEN,cap-color_PINK,cap-color_PURPLE,cap-color_RED,cap-color_WHITE,cap-color_YELLOW,bruises_BRUISES,bruises_NO,odor_ALMOND,odor_ANISE,odor_CREOSOTE,odor_FISHY,odor_FOUL,odor_MUSTY,odor_NONE,odor_PUNGENT,odor_SPICY,gill-attachment_ATTACHED,gill-attachment_FREE,gill-spacing_CLOSE,gill-spacing_CROWDED,gill-size_BROAD,gill-size_NARROW,gill-color_BLACK,gill-color_BROWN,gill-color_BUFF,gill-color_CHOCOLATE,gill-color_GRAY,gill-color_GREEN,gill-color_ORANGE,gill-color_PINK,gill-color_PURPLE,gill-color_RED,gill-color_WHITE,gill-color_YELLOW,stalk-shape_ENLARGING,stalk-shape_TAPERING,stalk-root_?,stalk-root_BULBOUS,stalk-root_CLUB,stalk-root_EQUAL,stalk-root_ROOTED,stalk-surface-above-ring_FIBROUS,stalk-surface-above-ring_SCALY,stalk-surface-above-ring_SILKY,stalk-surface-above-ring_SMOOTH,stalk-surface-below-ring_FIBROUS,stalk-surface-below-ring_SCALY,stalk-surface-below-ring_SILKY,stalk-surface-below-ring_SMOOTH,stalk-color-above-ring_BROWN,stalk-color-above-ring_BUFF,stalk-color-above-ring_CINNAMON,stalk-color-above-ring_GRAY,stalk-color-above-ring_ORANGE,stalk-color-above-ring_PINK,stalk-color-above-ring_RED,stalk-color-above-ring_WHITE,stalk-color-above-ring_YELLOW,stalk-color-below-ring_BROWN,stalk-color-below-ring_BUFF,stalk-color-below-ring_CINNAMON,stalk-color-below-ring_GRAY,stalk-color-below-ring_ORANGE,stalk-color-below-ring_PINK,stalk-color-below-ring_RED,stalk-color-below-ring_WHITE,stalk-color-below-ring_YELLOW,veil-type_PARTIAL,veil-color_BROWN,veil-color_ORANGE,veil-color_WHITE,veil-color_YELLOW,ring-number_NONE,ring-number_ONE,ring-number_TWO,ring-type_EVANESCENT,ring-type_FLARING,ring-type_LARGE,ring-type_NONE,ring-type_PENDANT,spore-print-color_BLACK,spore-print-color_BROWN,spore-print-color_BUFF,spore-print-color_CHOCOLATE,spore-print-color_GREEN,spore-print-color_ORANGE,spore-print-color_PURPLE,spore-print-color_WHITE,spore-print-color_YELLOW,population_ABUNDANT,population_CLUSTERED,population_NUMEROUS,population_SCATTERED,population_SEVERAL,population_SOLITARY,habitat_GRASSES,habitat_LEAVES,habitat_MEADOWS,habitat_PATHS,habitat_URBAN,habitat_WASTE,habitat_WOODS
0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [14]:
X_encoding.shape

(8416, 117)

In [16]:
# Verificação dos dados nulos do dataset
pd.set_option('display.max_rows', None)
nulos = X_encoding.isnull().sum()
percentual_nulos = (nulos / len(X_encoding)) * 100
print(percentual_nulos)

cap-shape_BELL                      0.0
cap-shape_CONICAL                   0.0
cap-shape_CONVEX                    0.0
cap-shape_FLAT                      0.0
cap-shape_KNOBBED                   0.0
cap-shape_SUNKEN                    0.0
cap-surface_FIBROUS                 0.0
cap-surface_GROOVES                 0.0
cap-surface_SCALY                   0.0
cap-surface_SMOOTH                  0.0
cap-color_BROWN                     0.0
cap-color_BUFF                      0.0
cap-color_CINNAMON                  0.0
cap-color_GRAY                      0.0
cap-color_GREEN                     0.0
cap-color_PINK                      0.0
cap-color_PURPLE                    0.0
cap-color_RED                       0.0
cap-color_WHITE                     0.0
cap-color_YELLOW                    0.0
bruises_BRUISES                     0.0
bruises_NO                          0.0
odor_ALMOND                         0.0
odor_ANISE                          0.0
odor_CREOSOTE                       0.0


In [19]:
y = y.map({'EDIBLE': 0, 'POISONOUS': 1})

In [20]:
y.value_counts()

,count
mushroom,
0,4488
1,3928


In [21]:
y.head()

,mushroom
0,0
1,0
2,0
3,0
4,0


## Aplicação do Modelo

In [23]:
# Criação do Modelo
modelo1 = ExtraTreesClassifier(n_jobs=-1)
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
resultado = cross_val_score(modelo1, X_encoding, y, cv=skfold, n_jobs=-1)
print(f'Acurácia média: {resultado.mean()}')

Acurácia média: 1.0


In [24]:
# Teste de outro modelo
modelo2 = BaggingClassifier(estimator=ExtraTreesClassifier(n_jobs=-1),
                            n_estimators=100, n_jobs=-1,
                            random_state=42)
resultado = cross_val_score(modelo2, X_encoding, y, cv=skfold, n_jobs=-1)
print(f'Acurácia média: {resultado.mean()}')

Acurácia média: 1.0


## Testando uma diferente aplicação

In [25]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': np.array([3, 5, 7, 9, 11, 13]),
    'min_samples_split': np.array([2, 4, 6, 8, 10, 12]),
    'min_samples_leaf': np.array([1, 3, 5, 7, 9, 11])
}

In [26]:
modelo3 = ExtraTreesClassifier(n_jobs=-1)
random_search = RandomizedSearchCV(modelo3, param_grid, cv=skfold, n_jobs=-1)
random_search.fit(X_encoding, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=ExtraTreesClassifier(n_jobs=-1), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': array([ 3,  5,  7,  9, 11, 13]),
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12])})

In [27]:
random_search.best_params_

{'min_samples_split': 8,
 'min_samples_leaf': 1,
 'max_depth': 7,
 'criterion': 'entropy'}

In [28]:
random_search.best_score_

1.0

## Outra Validação

In [29]:
X_tr, X_ts, y_tr, y_ts = train_test_split(X_encoding, y, test_size=0.3, random_state=42)

modelo4 = BaggingClassifier(estimator=ExtraTreesClassifier(min_samples_leaf=1,
                                                           min_samples_split=8,
                                                           max_depth=7,
                                                           criterion='entropy'),
                            n_estimators=100,
                            n_jobs=-1,
                            random_state=42)

modelo4.fit(X_tr, y_tr)

BaggingClassifier(estimator=ExtraTreesClassifier(criterion='entropy',
                                                 max_depth=7,
                                                 min_samples_split=8),
                  n_estimators=100, n_jobs=-1, random_state=42)

In [30]:
y_pred = modelo4.predict(X_ts)

acc = accuracy_score(y_ts, y_pred)
confusao = confusion_matrix(y_ts, y_pred)
classificacao = classification_report(y_ts, y_pred)

In [32]:
print(f"Acurácia: {acc}")

Acurácia: 1.0


In [33]:
print(f"Matriz de Confusão:\n{confusao}")

Matriz de Confusão:
[[1355    0]
 [   0 1170]]


In [34]:
print(f"Relatório de Classificação:\n{classificacao}")

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1355
           1       1.00      1.00      1.00      1170

    accuracy                           1.00      2525
   macro avg       1.00      1.00      1.00      2525
weighted avg       1.00      1.00      1.00      2525

